In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Uploading kaggle.json file

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ainuraa","key":"7bed69885a22c506d607b1c4ce58e4ff"}'}

Install the kaggle API

In [3]:
!pip install -q kaggle

Moving the kaggle.json file into ~/.kaggle

In [4]:
!mkdir -p ~/.kaggle

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [6]:
!cp kaggle.json ~/.kaggle/

Downloading files

In [7]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-dataset -p '/content'

100% 23.1G/23.1G [03:26<00:00, 158MB/s]
100% 23.1G/23.1G [03:26<00:00, 120MB/s]


In [ ]:
!unzip '/content/fashion-product-images-dataset.zip'

### Preprocessing

Checking the existing indices of images

In [9]:
import os 
path = '/content/fashion-dataset/images'

links = {}
for file in os.listdir(path):
    links[int(file.split('.')[0])] = path + '/'+ file

In [10]:
ids_images = list(links.keys())

In [46]:
len(ids_images)

44441

Reading the styles of each product

In [44]:
import os, json
import pandas as pd


path_to_json = '/content/fashion-dataset/fashion-dataset/styles/'
json_files = [json.load(open(path_to_json+pos_json)) for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

In [45]:
len(json_files)

44446

In [ ]:
# import re


# product_descriptions = []
# for style in json_files:
#     id = style['data']['id']
#     if id in ids_images:
#         dic = {}
#         data = style['data']
#         displayCategories = ''
#         if 'displayCategories' in data:
#           displayCategories = data['displayCategories'].split(',')[0]

#         name = data['productDisplayName'].replace(data['brandName'],"")+ ' ' + \
#               data['season'] + ' ' + data['usage'] + ' ' + displayCategories + ' '
#         if data['baseColour'] not in name:
#           name += data['baseColour']
#         name = re.sub(r"[.,\/#!$&%\^\*;:{}=\_`~()@0-9\[\]]", '', name)
#         name = re.sub(r'\s+', ' ', name).strip()
#         dic['id'] = id
#         dic['description'] = name
#         product_descriptions.append(dic)

In [52]:
import re


product_descriptions = []
for style in json_files:
    id = style['data']['id']
    category = style['data']['subCategory']['typeName']
    desired_category = 'Topwear'
    if id in ids_images:
        if category == desired_category and 'description' in style['data']['productDescriptors']:
            dic = {}
            data = style['data']
            name = data['productDescriptors']['description']['value']
            res = re.search(r'<br />(.*?)<br />', name)
            if not res:
              continue
            name = res.group(1)
            name = re.sub('<.*?>', '', name).strip()
            dic['id'] = id
            dic['description'] = name
            product_descriptions.append(dic)

In [57]:
with open('/content/drive/MyDrive/Github/Product-image-generation-from-text-description/descriptions_topwear.json', 'w') as fp:
    json.dump(product_descriptions, fp)